## Allowing Human Feedback in Agents

Currently AutoGen supports three modes for human input. The mode is specified through the human_input_mode argument of the ConversableAgent. The three modes are:

- NEVER: human input is never requested.
- TERMINATE (default): human input is only requested when a termination condition is met. Note that in this mode if the human chooses to intercept and reply, the conversation continues and the counter used by max_consectuive_auto_reply is reset.
- ALWAYS: human input is always requested and the human can choose to skip and trigger an auto-reply, intercept and provide feedback, or terminate the conversation. Note that in this mode termination based on max_consecutive_auto_reply is ignored.



1. Setup LLM

In [ ]:
from autogen import config_list_from_json

myModelname="GPT35TUEBOFIRST" #your model Name

#Read config File AOAI_CONFIG_LIST.json
config_list = config_list_from_json(
    env_or_file="..\AOAI_CONFIG_LIST.json",
    filter_dict={"model": myModelname}
    )

# LLM config file for myModelname (your model Name)

llm_myModelName_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }

## 2. Human Input Mode = NEVER

In this mode, human input is never requested and the termination conditions are used to terminate. This mode is useful when you want your agents to act fully autonomously.

In [ ]:
from autogen import ConversableAgent

agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. You have the "
    "number 53 in your mind, and I will try to guess it. "
    "If I guess too high, say 'too high', if I guess too low, say 'too low'. If I guess correctly, say 'correct it is 53'.",
    llm_config=llm_myModelName_config,
     is_termination_msg=lambda msg: "53" in msg["content"],  # terminate if the number is guessed by the other agent
    human_input_mode="NEVER",  # never ask for human input
)

agent_guess_number = ConversableAgent(
    "agent_guess_number",
    system_message="I have a number in my mind, and you will try to guess it. "
    "If I say 'too high', you should guess a lower number. If I say 'too low', "
    "you should guess a higher number. ",
    llm_config=llm_myModelName_config,
    human_input_mode="NEVER",
)

Play: example of using this mode to run a simple guess-a-number game between two agents, the termination message is set to check for the number that is the correct guess.

In [ ]:
result = agent_with_number.initiate_chat(
    agent_guess_number,
    message="I have a number between 1 and 100. Guess it!",
)

## 2. Human Input Mode = ALWAYS

In this mode, human input is always requested and the human can choose to skip, intersecpt, or terminate the conversation. Let us see this mode in action by playing the same game as before with the agent with the number, but this time participating in the game as a human. We will be the agent that is guessing the number, and play against the agent with the number from before.

In [ ]:
human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # no LLM used for human proxy
    human_input_mode="ALWAYS",  # always ask for human input
)


In [ ]:
# Start a chat with the agent with number with an initial guess.
result = human_proxy.initiate_chat(
    agent_with_number,  # this is the same agent with the number as before
    message="10",
)

## 3. Human Input Mode = TERMINATE

In this mode, human input is only requested when a termination condition is met. If the human choose to intercept and reply, the counter will be reset; if the human choose to skip, automatic reply mechanism will be used; if the human choose to terminate, the conversation will be terminated.

In [ ]:
agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. "
    "In the first game, you have the "
    "number 53 in your mind, and I will try to guess it. "
    "If I guess too high, say 'too high', if I guess too low, say 'too low'. "
    "If I said 53, you should respond: Congratulations, that's correct! The number I was thinking of is indeed 53. Well done!",
    llm_config=llm_myModelName_config,
    max_consecutive_auto_reply=1,  # maximum number of consecutive auto-replies before asking for human input
    is_termination_msg=lambda msg: "53" in msg["content"],  # terminate if the number is guessed by the other agent
    human_input_mode="TERMINATE",  # ask for human input until the game is terminated
)

In [ ]:
result = agent_with_number.initiate_chat(
    agent_guess_number,
    message="I have a number between 1 and 100. Guess it!",
)